# Problem 2(a)

2. Now, neglect the grid lines in the above problem. Find the trajectory from A (x,y,θ) = (3,15,0) to B
(x,y,θ) = (23,1,0) for the following system

$x = v cos θ $

$y = v sin θ $

$θ = u1 $
 
$v = u2 $
 
where θ is the angle with respect to the x axis, u1,u2 ∈ [−1,1]. You may neglect the dimension of the
car.

(a) Use RRT and shortest path search.

Some part of the code based on nimRobotics' code

link : https://github.com/nimRobotics/RRT

In [1]:
import cv2
import numpy as np
import math
import random

In [2]:
class Nodes:
    """Class to store the RRT graph"""
    def __init__(self, x,y):
        self.x = x
        self.y = y
        self.parent_x = []
        self.parent_y = []

In [3]:
map_img = cv2.imread("hw_map.png", cv2.IMREAD_COLOR)

In [4]:
occ_map = [[0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,1,1],
            [0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
            [0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
            [0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,1,1,1,0,0,0,1],
            [1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1],
            [1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1],
            [1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1],
            [1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,1,1],
            [1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1],
            [1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1],
            [1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1],
            [0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1],
            [0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1],
            [0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1],
            [0,0,0,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1]]

In [5]:
def is_occupied(x, y):
    """
    Check if the point (x, y) is occupied.

    Args:
        x (float): The pixel x-coordinate of the point
        y (float): The pixel y-coordinate of the point

    Returns:
        bool: True if the point is occupied, False otherwise.
    """

    img_h, img_w, _ = map_img.shape
    # Check if the coordinates are within the map boundaries
    if x < 0 or y < 0 or x > img_w or y > img_h:
        return True  # Consider out-of-bounds cells as occupied
    
    grid_h = len(occ_map) 
    grid_w = len(occ_map[0]) 
    # Convert float coordinates to integers (rounding to nearest integer)
    x_int = int(x/img_w * grid_w)
    y_int = int(y/img_h * grid_h)
    # Retrieve the occupancy status of the specified grid cell
    return occ_map[y_int][x_int] == 1

In [6]:
is_occupied(457, 241)

True

In [7]:
# check collision
def collision(x1,y1,x2,y2):
    if is_occupied(x1, y1) or is_occupied(x2, y2):
        return True  # Collision at endpoints
    # Divide the line into 100 points
    for i in range(1, 101):
        # Interpolate x and y coordinates
        x = x1 + (x2 - x1) * i / 100
        y = y1 + (y2 - y1) * i / 100
        # Check for collision at the interpolated point
        if is_occupied(x, y):
            return True  # Collision detected

    return False  # No collision detected

In [8]:
collision(0,0,120,120)

False

In [9]:
# return dist and angle b/w new point and nearest node
def dist_and_angle(x1,y1,x2,y2):
    dist = math.sqrt( ((x1-x2)**2)+((y1-y2)**2) )
    angle = math.atan2(y2-y1, x2-x1)
    return(dist,angle)

In [10]:
# check the  collision with obstacle and trim
def check_collision(k,goal,x1,y1,x2,y2):
    dist,theta = dist_and_angle(x2,y2,x1,y1)
    x=x2 + k*dist*np.cos(theta)
    y=y2 + k*dist*np.sin(theta)
    # print(x2,y2,x1,y1)
    # print("theta",theta)
    # print("check_collision",x,y)

    # TODO: trim the branch if its going out of image area
    h,w, _=map_img.shape
    if y<0 or y>h or x<0 or x>w:
        print("Point out of image bound")
        directCon = False
        nodeCon = False
    else:
        # check direct connection
        if collision(x,y,goal[0],goal[1]):
            directCon = False
        else:
            directCon=True

        # check connection between two nodes
        if collision(x,y,x2,y2):
            nodeCon = False
        else:
            nodeCon = True

    return(x,y,directCon,nodeCon)

In [11]:
# return the neaerst node index
def nearest_node(node_list,x,y):
    temp_dist=[]
    for i in range(len(node_list)):
        dist,_ = dist_and_angle(x,y,node_list[i].x,node_list[i].y)
        temp_dist.append(dist)
    return temp_dist.index(min(temp_dist))

In [12]:
# generate a random point in the image space
def random_point(h,l):
    new_y = random.randint(0, h)
    new_x = random.randint(0, l)
    return (new_x,new_y)

In [13]:
def RRT(map_img):
    h,l,_= map_img.shape # dim of the loaded image
    start = (72,18)
    goal = (620,463)
    node_list = [0]
    k = 0.2
    # insert the starting point in the node class
    # node_list = [0] # list to store all the node points         
    node_list[0] = Nodes(start[0],start[1])
    node_list[0].parent_x.append(start[0])
    node_list[0].parent_y.append(start[1])
    # display start and goal
    cv2.circle(map_img, (start[0],start[1]),radius=3,color=(0,0,0), thickness=-1)
    cv2.circle(map_img, (goal[0],goal[1]),radius=3,color=(0,0,0), thickness=-1)

    i=1
    pathFound = False
    while pathFound==False:
        nx,ny = random_point(h,l)
        print("Random points:",nx,ny)
        nearest_ind = nearest_node(node_list,nx,ny)
        nearest_x = node_list[nearest_ind].x
        nearest_y = node_list[nearest_ind].y
        print("Nearest node coordinates:",nearest_x,nearest_y)

        #check direct connection
        tx,ty,directCon,nodeCon = check_collision(k,goal,nx,ny,nearest_x,nearest_y)
        print("Check collision:",tx,ty,directCon,nodeCon)

        if nodeCon:
            node_list.append(i)
            node_list[i] = Nodes(tx,ty)
            node_list[i].parent_x = node_list[nearest_ind].parent_x.copy()
            node_list[i].parent_y = node_list[nearest_ind].parent_y.copy()
            node_list[i].parent_x.append(tx)
            node_list[i].parent_y.append(ty)

            cv2.circle(map_img, (int(tx),int(ty)),radius=3,color=(0,0,0), thickness=-1)
            cv2.line(map_img, (int(tx),int(ty)), (int(node_list[nearest_ind].x),int(node_list[nearest_ind].y)), (0,0,0), thickness=1, lineType=8)
            
            if directCon:
                print("Node can connect directly with end")
                cv2.line(map_img, (int(tx),int(ty)), (goal[0],goal[1]), (255,0,0), thickness=2, lineType=8)

                cv2.imshow("RRT",map_img)
                for j in range(len(node_list[i].parent_x)-1):
                    cv2.line(map_img, (int(node_list[i].parent_x[j]),int(node_list[i].parent_y[j])), (int(node_list[i].parent_x[j+1]),int(node_list[i].parent_y[j+1])), (255,0,0), thickness=2, lineType=8)
                
                cv2.imshow("RRT",map_img)
                cv2.imwrite("out.jpg",map_img)
                cv2.destroyWindow("My Window")
                pathFound = True
                return
            
            else:
                i+=1
                # display
                cv2.imshow("RRT",map_img)
                cv2.waitKey(1)
                continue

        else:
            print("No direct con. and no node con. :( Generating new rnd numbers")
            continue

    

In [14]:
raw_map_img = cv2.imread("hw_map.png", cv2.IMREAD_COLOR)

RRT(raw_map_img)

Random points: 462 90
Nearest node coordinates: 72 18
Check collision: 150.0 32.4 False False
No direct con. and no node con. :( Generating new rnd numbers
Random points: 180 464
Nearest node coordinates: 72 18
Check collision: 93.6 107.20000000000002 False True
Random points: 271 432
Nearest node coordinates: 93.6 107.20000000000002
Check collision: 129.07999999999998 172.16000000000003 False False
No direct con. and no node con. :( Generating new rnd numbers
Random points: 527 415
Nearest node coordinates: 93.6 107.20000000000002
Check collision: 180.27999999999997 168.76000000000002 False False
No direct con. and no node con. :( Generating new rnd numbers
Random points: 242 26
Nearest node coordinates: 93.6 107.20000000000002
Check collision: 123.28 90.96000000000001 False True
Random points: 589 70
Nearest node coordinates: 123.28 90.96000000000001
Check collision: 216.42400000000004 86.768 False False
No direct con. and no node con. :( Generating new rnd numbers
Random points: 296

2024-04-12 23:03:11.614 Python[11299:409011] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


Random points: 269 359
Nearest node coordinates: 115.04416 288.28832
Check collision: 145.835328 302.430656 False True
Random points: 454 46
Nearest node coordinates: 125.224 80.56800000000001
Check collision: 190.9792 73.65440000000001 False False
No direct con. and no node con. :( Generating new rnd numbers
Random points: 483 210
Nearest node coordinates: 145.835328 302.430656
Check collision: 213.2682624 283.9445248 False True
Random points: 528 30
Nearest node coordinates: 213.2682624 283.9445248
Check collision: 276.21460992 233.15561983999999 False True
Random points: 376 408
Nearest node coordinates: 276.21460992 233.15561983999999
Check collision: 296.171687936 268.124495872 False True
Random points: 399 347
Nearest node coordinates: 296.171687936 268.124495872
Check collision: 316.7373503488 283.89959669760003 False True
Random points: 174 197
Nearest node coordinates: 101.944 213.888
Check collision: 116.3552 210.5104 False False
No direct con. and no node con. :( Generating 

In [15]:
print(map_img.shape)

(477, 688, 3)
